<a href="https://colab.research.google.com/github/Abhiramsaigoud/2203A51272-B18/blob/main/Assignment_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Use a simple dataset for English-to-French translation. You can either use a small dataset like this or download a more extensive dataset such as the Tab-delimited Bilingual Sentence Pairs dataset from Tatoeba or Parallel Corpus from the European Parliament.
# Example data (small English to French pairs)

data = [ ("hello", "bonjour"), ("how are you", "comment ça va"), ("I am fine", "je vais bien"), ("what is your name", "comment tu t'appelles"), ("my name is", "je m'appelle"), ("thank you", "merci"), ("goodbye", "au revoir") ]


(a) Data Preprocessing

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Example dataset (small English to French pairs)
data = [("hello", "bonjour"),
        ("how are you", "comment ça va"),
        ("I am fine", "je vais bien"),
        ("what is your name", "comment tu t'appelles"),
        ("my name is", "je m'appelle"),
        ("thank you", "merci"),
        ("goodbye", "au revoir")]

# Separate into English and French sentences
english_sentences = [pair[0] for pair in data]
french_sentences = [pair[1] for pair in data]

# Tokenize the English and French sentences
english_tokenizer = Tokenizer()
french_tokenizer = Tokenizer()

english_tokenizer.fit_on_texts(english_sentences)
french_tokenizer.fit_on_texts(french_sentences)

# Convert text to sequences of integers
english_sequences = english_tokenizer.texts_to_sequences(english_sentences)
french_sequences = french_tokenizer.texts_to_sequences(french_sentences)

# Pad the sequences to have uniform length
max_english_seq_len = max([len(seq) for seq in english_sequences])
max_french_seq_len = max([len(seq) for seq in french_sequences])

english_padded = pad_sequences(english_sequences, maxlen=max_english_seq_len, padding='post')
french_padded = pad_sequences(french_sequences, maxlen=max_french_seq_len, padding='post')

# Get the size of the vocabularies
english_vocab_size = len(english_tokenizer.word_index) + 1
french_vocab_size = len(french_tokenizer.word_index) + 1

# Output
print("English Sentences (Original Text):")
print(english_sentences)

print("\nFrench Sentences (Original Text):")
print(french_sentences)

print("\nTokenized English Sequences:")
print(english_sequences)

print("\nTokenized French Sequences:")
print(french_sequences)

print("\nPadded English Sequences:")
print(english_padded)

print("\nPadded French Sequences:")
print(french_padded)

print(f"\nEnglish Vocabulary Size: {english_vocab_size}")
print(f"French Vocabulary Size: {french_vocab_size}")


English Sentences (Original Text):
['hello', 'how are you', 'I am fine', 'what is your name', 'my name is', 'thank you', 'goodbye']

French Sentences (Original Text):
['bonjour', 'comment ça va', 'je vais bien', "comment tu t'appelles", "je m'appelle", 'merci', 'au revoir']

Tokenized English Sequences:
[[4], [5, 6, 1], [7, 8, 9], [10, 2, 11, 3], [12, 3, 2], [13, 1], [14]]

Tokenized French Sequences:
[[3], [1, 4, 5], [2, 6, 7], [1, 8, 9], [2, 10], [11], [12, 13]]

Padded English Sequences:
[[ 4  0  0  0]
 [ 5  6  1  0]
 [ 7  8  9  0]
 [10  2 11  3]
 [12  3  2  0]
 [13  1  0  0]
 [14  0  0  0]]

Padded French Sequences:
[[ 3  0  0]
 [ 1  4  5]
 [ 2  6  7]
 [ 1  8  9]
 [ 2 10  0]
 [11  0  0]
 [12 13  0]]

English Vocabulary Size: 15
French Vocabulary Size: 14


(b) Build Seq2Seq Model

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense

# Define model parameters
embedding_dim = 64
lstm_units = 128

# Encoder
encoder_inputs = Input(shape=(max_english_seq_len,))
encoder_embedding = Embedding(english_vocab_size, embedding_dim)(encoder_inputs)
encoder_lstm, encoder_state_h, encoder_state_c = LSTM(lstm_units, return_state=True)(encoder_embedding)
encoder_states = [encoder_state_h, encoder_state_c]

# Decoder
decoder_inputs = Input(shape=(max_french_seq_len,))
decoder_embedding = Embedding(french_vocab_size, embedding_dim)(decoder_inputs)
decoder_lstm, _, _ = LSTM(lstm_units, return_sequences=True, return_state=True)(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(french_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_lstm)

# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 4)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_1             │ (None, 3)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding (Embedding)     │ (None, 4, 64)          │            960 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_1 (Embedding)   │ (None, 3, 64)          │            896 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm (LSTM)               │ [(None, 128), (None,   │         98,816 │ embedding[0][0]        │
│                           │ 128), (None, 128)]     │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_1 (LSTM)             │ [(None, 3, 128),       │         98,816 │ embedding_1[0][0],     │
│                           │ (None, 128), (None,    │                │ lstm[0][1], lstm[0][2] │
│                           │ 128)]                  │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 3, 14)          │          1,806 │ lstm_1[0][0]           │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 201,294 (786.30 KB)

 Trainable params: 201,294 (786.30 KB)

 Non-trainable params: 0 (0.00 B)

(c) Preparing the Data for Training

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Example dataset (small English to French pairs)
data = [("hello", "bonjour"),
        ("how are you", "comment ça va"),
        ("I am fine", "je vais bien"),
        ("what is your name", "comment tu t'appelles"),
        ("my name is", "je m'appelle"),
        ("thank you", "merci"),
        ("goodbye", "au revoir")]

# Separate into English and French sentences
english_sentences = [pair[0] for pair in data]
french_sentences = [pair[1] for pair in data]

# Tokenize the English and French sentences
english_tokenizer = Tokenizer()
french_tokenizer = Tokenizer()

english_tokenizer.fit_on_texts(english_sentences)
french_tokenizer.fit_on_texts(french_sentences)

# Convert text to sequences of integers
english_sequences = english_tokenizer.texts_to_sequences(english_sentences)
french_sequences = french_tokenizer.texts_to_sequences(french_sentences)

# Pad the sequences to have uniform length
max_english_seq_len = max([len(seq) for seq in english_sequences])
max_french_seq_len = max([len(seq) for seq in french_sequences])

english_padded = pad_sequences(english_sequences, maxlen=max_english_seq_len, padding='post')
french_padded = pad_sequences(french_sequences, maxlen=max_french_seq_len, padding='post')

# Prepare decoder input and output data
decoder_input_data = french_padded[:, :-1]
decoder_output_data = french_padded[:, 1:]

# Expand decoder output to 3D shape for sparse_categorical_crossentropy
decoder_output_data = np.expand_dims(decoder_output_data, -1)

# Output
print("Decoder Input Data:")
print(decoder_input_data)

print("\nDecoder Output Data (3D):")
print(decoder_output_data)


Decoder Input Data:
[[ 3  0]
 [ 1  4]
 [ 2  6]
 [ 1  8]
 [ 2 10]
 [11  0]
 [12 13]]

Decoder Output Data (3D):
[[[ 0]
  [ 0]]

 [[ 4]
  [ 5]]

 [[ 6]
  [ 7]]

 [[ 8]
  [ 9]]

 [[10]
  [ 0]]

 [[ 0]
  [ 0]]

 [[13]
  [ 0]]]


(d) Train the model on the dataset

In [ ]:
# Train the model
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense

# Example dataset (small English to French pairs)
data = [("hello", "bonjour"),
        ("how are you", "comment ça va"),
        ("I am fine", "je vais bien"),
        ("what is your name", "comment tu t'appelles"),
        ("my name is", "je m'appelle"),
        ("thank you", "merci"),
        ("goodbye", "au revoir")]

# Separate into English and French sentences
english_sentences = [pair[0] for pair in data]
french_sentences = [pair[1] for pair in data]

# Tokenize the English and French sentences
english_tokenizer = Tokenizer()
french_tokenizer = Tokenizer()

english_tokenizer.fit_on_texts(english_sentences)
french_tokenizer.fit_on_texts(french_sentences)

# Convert text to sequences of integers
english_sequences = english_tokenizer.texts_to_sequences(english_sentences)
french_sequences = french_tokenizer.texts_to_sequences(french_sentences)

# Pad the sequences to have uniform length
max_english_seq_len = max([len(seq) for seq in english_sequences])
max_french_seq_len = max([len(seq) for seq in french_sequences])

english_padded = pad_sequences(english_sequences, maxlen=max_english_seq_len, padding='post')
french_padded = pad_sequences(french_sequences, maxlen=max_french_seq_len, padding='post')

# Prepare decoder input and output data
decoder_input_data = french_padded[:, :-1]
decoder_output_data = french_padded[:, 1:]

# Expand decoder output to 3D shape for sparse_categorical_crossentropy
decoder_output_data = np.expand_dims(decoder_output_data, -1)

# Define the parameters for the model
embedding_dim = 128
units = 256

# Define the encoder
encoder_inputs = Input(shape=(None,))  # English input shape
encoder_embedding = Embedding(input_dim=len(english_tokenizer.word_index) + 1, output_dim=embedding_dim)(encoder_inputs)
encoder_lstm = LSTM(units, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

# Define the decoder
decoder_inputs = Input(shape=(None,))  # French input shape
decoder_embedding = Embedding(input_dim=len(french_tokenizer.word_index) + 1, output_dim=embedding_dim)(decoder_inputs)
decoder_lstm = LSTM(units, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(len(french_tokenizer.word_index) + 1, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    [english_padded, decoder_input_data],
    decoder_output_data,
    batch_size=2,  # You can adjust the batch size
    epochs=10,    # Number of epochs for training
    validation_split=0.2,
    verbose=1      # Verbosity level
)

# Output the final metrics
print("Final Training Loss:", history.history['loss'][-1])
print("Final Training Accuracy:", history.history['accuracy'][-1])
print("Final Validation Loss:", history.history['val_loss'][-1])
print("Final Validation Accuracy:", history.history['val_accuracy'][-1])


Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 8s 606ms/step - accuracy: 0.0000e+00 - loss: 2.6423 - val_accuracy: 0.7500 - val_loss: 2.5994
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step - accuracy: 0.6125 - loss: 2.5885 - val_accuracy: 0.7500 - val_loss: 2.5368
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step - accuracy: 0.5938 - loss: 2.5211 - val_accuracy: 0.7500 - val_loss: 2.4322
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step - accuracy: 0.3438 - loss: 2.4785 - val_accuracy: 0.7500 - val_loss: 2.3084
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.4500 - loss: 2.3066 - val_accuracy: 0.7500 - val_loss: 2.0669
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.2438 - loss: 2.2006 - val_accuracy: 0.7500 - val_loss: 1.7630
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.4313 - loss: 1.8160 - val_accuracy: 0.7500 - val_loss: 1.4988
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.2125 - loss: 1.8765 - val_accuracy: 0.7500 - val_loss: 1

(e) Inference Setup for Translation

In [ ]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, TimeDistributed
from tensorflow.keras.models import Model

# Define the units and input shapes
lstm_units = 256  # The number of LSTM units you defined earlier

# Encoder inference model
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder inference model
decoder_state_input_h = Input(shape=(lstm_units,))
decoder_state_input_c = Input(shape=(lstm_units,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# Use the decoder embedding layer from the training model
decoder_embedding_inf = Embedding(input_dim=len(french_tokenizer.word_index) + 1, output_dim=embedding_dim)(decoder_inputs)

# Pass the embedding through the LSTM
decoder_lstm_inf, decoder_state_h_inf, decoder_state_c_inf = LSTM(lstm_units, return_sequences=True, return_state=True)(
    decoder_embedding_inf, initial_state=decoder_states_inputs)

# Connect the Dense layer to the LSTM output
decoder_outputs_inf = decoder_dense(decoder_lstm_inf)

# Create the decoder model
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs_inf] + [decoder_state_h_inf, decoder_state_c_inf])

# Print the model summaries
encoder_model.summary()
decoder_model.summary()


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)           │ (None, None)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_4 (Embedding)              │ (None, None, 128)           │           1,920 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_4 (LSTM)                        │ [(None, 256), (None, 256),  │         394,240 │
│                                      │ (None, 256)]                │                 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 396,160 (1.51 MB)

 Trainable params: 396,160 (1.51 MB)

 Non-trainable params: 0 (0.00 B)

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5             │ (None, None)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_6 (Embedding)   │ (None, None, 128)      │          1,792 │ input_layer_5[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_8             │ (None, 256)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_9             │ (None, 256)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_7 (LSTM)             │ [(None, None, 256),    │        394,240 │ embedding_6[0][0],     │
│                           │ (None, 256), (None,    │                │ input_layer_8[0][0],   │
│                           │ 256)]                  │                │ input_layer_9[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, None, 14)       │          3,598 │ lstm_7[0][0]           │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 399,630 (1.52 MB)

 Trainable params: 399,630 (1.52 MB)

 Non-trainable params: 0 (0.00 B)

(f) Translate New Sentences

In [ ]:
# Function to decode sequence (translation)
def decode_sequence(input_seq):
    # Encode the input as state vectors
    states_value = encoder_model.predict(input_seq)

    # Generate an empty target sequence of length 1, initialized with the start token index
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = french_tokenizer.word_index.get('<start>', 0)  # Ensure this is a valid start token

    stop_condition = False
    decoded_sentence = ''

    while not stop_condition:
        # Predict the next token
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Get the predicted token index
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = french_tokenizer.index_word.get(sampled_token_index, '')  # Safe access

        # Check if the sampled word is defined
        if sampled_word:  # Only add if the word is valid
            decoded_sentence += ' ' + sampled_word

        # Check for stopping condition
        if sampled_word == '<end>' or len(decoded_sentence.split()) > max_french_seq_len:  # Use the actual end token
            stop_condition = True

        # Update the target sequence (of length 1)
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence.strip()  # Remove leading space

# Example translation
test_sentence = "hello"
test_sequence = english_tokenizer.texts_to_sequences([test_sentence])
test_padded = pad_sequences(test_sequence, maxlen=max_english_seq_len, padding='post')
translated_sentence = decode_sequence(test_padded)
print("Translated sentence:", translated_sentence)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 409ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Translated sentence: t'appelles t'appelles t'appelles t'appelles
